In [1]:
from pathlib import Path
from datetime import datetime, timedelta

def average_latency(sub_log: Path) -> float:
	readings = []
	with sub_log.open() as f:
		for line in f:
			if " - " in line:
				ts, val = line.split(" - ")
				readings.append((datetime.fromisoformat(ts), datetime.fromisoformat(val.strip())))

	# remove last value (is unusally large latency due to timeout)
	readings = readings[:-2]

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	# add 1s to values (cache flushing issue)
	readings = [(ts, val + timedelta(seconds=1))  for ts, val, in readings]


	latency = [(ts-val).total_seconds() for ts, val in readings]

	return sum(latency)/len(latency)

def average_cpu_usage(relay_perf: Path) -> float:
	readings = []
	with relay_perf.open() as f:
		for line in f:
			ts, val = line.split("   ")
			if val == "":
				raise ValueError("empty val")
			readings.append((datetime.fromisoformat(ts.strip()[:-1]), float(val.strip())))

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	percentages = [v for _, v in readings]

	return sum(percentages)/len(percentages)


def load_measurements(dir: Path):
	measurements = dir / Path("measurements")
	if not measurements.is_dir():
		raise ValueError(f"invalid directory: {measurements}")

	sub_logs = []
	relay_perf = []

	# organize files
	for file in measurements.iterdir():
		if file.name.startswith("clock"):
			sub_logs.append(file)
		if file.name.endswith("cpu_usage.log"):
			relay_perf.append(file)

	sub_avg_latencies = []
	for sub in sub_logs:
		try:
			sub_avg_latencies.append(average_latency(sub))
		except Exception as e:
			pass#print(f"{sub} skipped due to {e}")

	relay_perf_avg = {}
	for relay in relay_perf:
		name = relay.stem.replace("_cpu_usage", "")
		try:
			relay_perf_avg[name] = average_cpu_usage(relay)
		except Exception as e:
			print(f"{name} skipped due to {e}")


	return sub_avg_latencies, relay_perf_avg

load_measurements(Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement/star"))

([0.017943634615384618,
  0.01894119607843137,
  0.018668960784313727,
  0.01871717647058824,
  0.010725862745098041,
  0.010186538461538462,
  0.01884401960784314,
  0.010059346153846153,
  0.01936260784313726,
  0.018294725490196072,
  0.018525294117647063,
  0.018982529411764704,
  0.018812294117647062,
  0.019485568627450985,
  0.019277980392156863,
  0.010219173076923077,
  0.010459173076923074,
  0.01967050980392157,
  0.019893411764705878,
  0.01885261538461538,
  0.017941942307692308,
  0.01921749019607843,
  0.019113999999999996,
  0.018563980769230774,
  0.019811862745098035,
  0.018781470588235295,
  0.019382627450980395,
  0.010868568627450979,
  0.019352607843137255,
  0.010618846153846156,
  0.010840137254901962,
  0.01873378846153847,
  0.019025058823529412,
  0.018733725490196078,
  0.009610666666666665,
  0.019891745098039215,
  0.018538057692307694,
  0.019550450980392157,
  0.018860019230769232,
  0.01794780392156863,
  0.019093450980392154,
  0.018571843137254903,
 

In [2]:
from pprint import pprint
base = Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement")

measurements = []
for topo in ["full-mesh", "star", "leaf-and-spine", "two-relays", "single-relay"]:
	latencies, relays = load_measurements(base / topo)

	avg_lat = sum(latencies) / len(latencies)
	topo_results = {
		"name": topo,
		"average_latency": avg_lat,
		"relay_utilizations": relays
	}
	measurements.append(topo_results)
	pprint(topo_results)

{'average_latency': 0.014831366673265461,
 'name': 'full-mesh',
 'relay_utilizations': {'relay1': 0.436734693877551,
                        'relay2': 0.41836734693877553,
                        'relay3': 0.4040816326530611,
                        'relay4': 0.3775510204081632,
                        'relay5': 0.39591836734693875}}
{'average_latency': 0.017165031517722487,
 'name': 'star',
 'relay_utilizations': {'hub': 0.3346938775510204,
                        'relay1': 0.39591836734693875,
                        'relay2': 0.3428571428571428,
                        'relay3': 0.3489795918367347,
                        'relay4': 0.3897959183673469,
                        'relay5': 0.3367346938775511}}
{'average_latency': 0.015889892295615257,
 'name': 'leaf-and-spine',
 'relay_utilizations': {'leaf1': 0.4387755102040818,
                        'leaf2': 0.4040816326530611,
                        'leaf3': 0.4061224489795918,
                        'leaf4': 0.4428571428571429,
 

In [3]:
for m in measurements:
    print(f"{m['name']}: {round(m['average_latency'] * 1000, 2)}")

full-mesh: 14.83
star: 17.17
leaf-and-spine: 15.89
two-relays: 16.29
single-relay: 17.4


In [4]:
# scale CPU utilization to highest
highest_cpu = max(max(t["relay_utilizations"].values()) for t in measurements)

for t in measurements:
	t["average_latency"] = f"{t['average_latency'] * 1000} ms"
	for r, val in t["relay_utilizations"].items():
		t["relay_utilizations"][r] = f"{val / highest_cpu:.0%}"
pprint(measurements)

[{'average_latency': '14.83136667326546 ms',
  'name': 'full-mesh',
  'relay_utilizations': {'relay1': '24%',
                         'relay2': '23%',
                         'relay3': '22%',
                         'relay4': '21%',
                         'relay5': '22%'}},
 {'average_latency': '17.165031517722486 ms',
  'name': 'star',
  'relay_utilizations': {'hub': '18%',
                         'relay1': '22%',
                         'relay2': '19%',
                         'relay3': '19%',
                         'relay4': '21%',
                         'relay5': '19%'}},
 {'average_latency': '15.889892295615256 ms',
  'name': 'leaf-and-spine',
  'relay_utilizations': {'leaf1': '24%',
                         'leaf2': '22%',
                         'leaf3': '22%',
                         'leaf4': '24%',
                         'leaf5': '23%',
                         'spine1': '14%',
                         'spine2': '8%'}},
 {'average_latency': '16.28975929015838 m